In [2]:
from urllib.parse import urlparse, urljoin, unquote
import pandas as pd
import wikipedia
from bs4 import BeautifulSoup
from fastcore.parallel import *
from functools import partial
wikipedia.set_lang('or')


In [3]:
import logging
import traceback

def setup_logging(log_file: str = "app.log"):
    # Create a custom logger
    logger = logging.getLogger("LinkExtractor")
    logger.setLevel(logging.INFO)  # Capture only INFO and above (INFO, WARNING, ERROR, CRITICAL)

    # Prevent logger duplication
    if not logger.hasHandlers():
        # Create file handler to log to a file
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)  # File handler also captures only INFO and above

        # Create log format with function name included
        log_format = logging.Formatter('%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s')
        file_handler.setFormatter(log_format)

        # Add handler to the logger
        logger.addHandler(file_handler)

    return logger

# Step 2: Set up logging to a file only, suppressing output in the notebook
logger = setup_logging(log_file="notebook_logs.log")

# Step 3: Example usage
logger.info("This is an INFO log message.")
logger.debug("This DEBUG message won't appear in INFO mode.")
logger.warning("This is a WARNING log message.")

In [4]:
!ls

Untitled-1.ipynb   get_all_odia_web_pages.ipynb  process_wikipedia_web2md.ipynb
content_links.csv  notebook_logs.log		 temp.md


In [5]:
df = pd.read_csv('content_links.csv')
df.head

<bound method NDFrame.head of                                                    links
0               https://or.wikipedia.org/wiki/0_(number)
1                      https://or.wikipedia.org/wiki/100
2                     https://or.wikipedia.org/wiki/1000
3                     https://or.wikipedia.org/wiki/1001
4                     https://or.wikipedia.org/wiki/1002
...                                                  ...
38106  https://or.wikipedia.org/wiki/‌‌କେନ୍ଦ୍ରାପଡ଼ା_ଜ...
38107      https://or.wikipedia.org/wiki/‌‌ନଗର,_ରାଜସ୍ଥାନ
38108        https://or.wikipedia.org/wiki/‌‌ନଗରକୁର୍ନୁଲ୍
38109  https://or.wikipedia.org/wiki/‌‌ନାଗରୀ,_ଆନ୍ଧ୍ରପ...
38110        https://or.wikipedia.org/wiki/﻿﻿Abelisaurus

[38111 rows x 1 columns]>

In [6]:
pg = urlparse(df.iloc[0]['links']).path.split('/wiki/')[1]
pg

'0_(number)'

In [7]:
ignore_txts = [
    'ଏହି ଲେଖାଟି ବଟ୍ ବ୍ୟବହାର କରି ତିଆରି କରାଯାଇଛି ଏବଂ ଏହା ପର୍ଯ୍ୟାୟ କ୍ରମେ ଉନ୍ନତି ହେଉଅଛି । ଏଣୁ ଏହି ଛାଞ୍ଚ ନଉଠିବା ପର୍ଯ୍ୟନ୍ତ କୌଣସି ବଦଳ କରନ୍ତୁ ନାହିଁ । ଯଦି ଆପଣ କୌଣସି ଗୁରୁତ୍ତ୍ୱପୂର୍ଣ୍ଣ ତ୍ରୁଟି ଆପଣଙ୍କ ଦୃଷ୍ଟିଗୋଚର ହେଉଅଛି ତେବେ ପୃଷ୍ଠା ଆଲୋଚନା‌ରେ ସୂଚିତ କରନ୍ତୁ ।',
    """୧ ଘଟଣା

୧.୧ ଜାନୁଆରୀ-ମାର୍ଚ୍ଚ
୧.୨ ଅପ୍ରେଲ-ଜୁନ
""", # for removing page like https://or.wikipedia.org/wiki/%E0%AD%A7%E0%AD%A6%E0%AD%A6
    ]

In [87]:
import re
from functools import reduce

# Define patterns for removing markdown and wrapped substrings
MARKDOWN_PATTERNS = [
    (re.compile(r'==+.*?==+', re.MULTILINE), ''),  # Remove substrings wrapped in === or ==    
    (re.compile(r'\s+\{.*\}\s*', re.DOTALL), ''),  # Remove LaTeX-style expressions

    (re.compile(r'\n[A-Za-z0-9,.\'\"!?;:()⋅−\-\{\}= \s]+\n', re.MULTILINE), ''),  # Remove lines with only English text
    (re.compile(r'[\n\t]+'), r'\n'),  # Remove multiple newlines

]



def remove_markdown(text, log_steps=False):
    if log_steps  : print(f"Original Text:\n{text}")
    
    def apply_and_print(text, pattern, log_steps=False):
        updated_text = pattern[0].sub(pattern[1], text)
        if log_steps  : print(f"After applying pattern {pattern[0].pattern}:\n{updated_text}\n") 
        return updated_text
    
    # Apply all the patterns defined in MARKDOWN_PATTERNS and print after each substitution
    final_text = reduce(lambda t, pattern: apply_and_print(t, pattern), MARKDOWN_PATTERNS, text)
    
    if log_steps  :  print(f"Final Text:\n{final_text}")
    return final_text


In [88]:
remove_markdown('\nIts SI units are newtons per coulomb (N⋅C−1) or, equivalently, volts per metre (V⋅m−1), which in terms of SI base units are kg⋅m⋅s−3⋅A−1.\n')

''

In [14]:
def all_strings_in_base(txt):
    """
    takes in html string and check if ignore_texts is present 
    by using beutifulsoup4
    """
    soup = BeautifulSoup(txt, 'html.parser')
    txt =  soup.get_text()
    
    return any(substr in txt for substr in ignore_txts)

def find_ref_pos(txt):
    #print(f"{txt=}")
    return txt[:txt.find('== ଆଧାର ==')]
    
def extract_txt(pg):
    try:
        page = wikipedia.page(pg)


        #check if given page have ignore_txt
        if all_strings_in_base(page.html()):
            return ''
        

        content = page.content

        #find the index of '== ଆଧାର =='
        if '== ଆଧାର ==' in content:
            content = find_ref_pos(content)

        # for removing '\n\n\n== ଘଟଣା ==\n\n\n==='
        if '==' in content:
            content = remove_markdown(content)
        return content
    except Exception as e:
        logger.error(f"Error getting following siblings: {str(e)}")
        logger.error(traceback.format_exc())
        return "!!!ERROR!!!"
        


As everthing after `== ଆଧାର ==` is like external links and refernece so removing those.

In [89]:
extract_txt('1130')

''

In [90]:
print(extract_txt('11p_deletion_syndrome'))

୧୧ପି ବିଲୋପନ ସିଣ୍ଡ୍ରୋମ, ପୂର୍ବ ନାମ ଡବ୍ଲ୍ୟୁଏଜିଆର ସିଣ୍ଡ୍ରୋମ (WAGR ସିଣ୍ଡ୍ରୋମ), ଏକ ଜେନେଟିକ ରୋଗ ଯେଉଁଥିରେ ଲୋକମାନେ ୱିମସ ଅର୍ବୁଦ (ଏକ ପ୍ରକାର ବୃକ୍‌କ କର୍କଟ ), ଆନିରିଡିଆ (ଆଖିର ରଙ୍ଗୀନ ଅଂଶର ଅନୁପସ୍ଥିତି), ଯୌନମୂତ୍ରାଙ୍ଗ ଅସ୍ୱାଭାବିକତା ଯେପରିକି ଅଣଅବରୋହୀ ଟେଷ୍ଟିସ ଓ ମାନସିକ ଅନ‌ଗ୍ରସରତା ରୋଗମାନଙ୍କର ସଙ୍କଟରେ  ଥାଆନ୍ତି । ଅନ୍ୟାନ୍ୟ ଲକ୍ଷଣ ମଧ୍ୟରେ ସାମାନ୍ୟ ଆଲୋକ ଗ୍ରହଣ, ମୋତିଆବିନ୍ଦୁ, ବାଲ୍ୟ ମୋଟାପଣ, ଅଗ୍ନାଶୟ ପ୍ରଦାହ ଏବଂ ବୃକ୍‌କ ବିଫଳତା  ଇତ୍ୟାଦି ଅନ୍ତର୍ଭୁକ୍ତ ହୋଇପାରେ ।  ବୌଦ୍ଧିକ ଅକ୍ଷମତା ସାଧାରଣତଃ ସାମାନ୍ୟରୁ ମଧ୍ୟମ ଶ୍ରେଣୀ ମଧ୍ୟରେ ଥାଏ । 
ଏହାର କାରଣ ହେଉଛି କ୍ରୋମୋଜୋମର ୧୧ତମ କ୍ଷୁଦ୍ର ବାହୁରେ ଜେନେଟିକ୍ ପଦାର୍ଥ ବିଲୋପ ସାଧନ ।  ପ୍ରାରମ୍ଭିକ ବିକାଶ ସମୟରେ ଅଧିକାଂଶ ମାମଲା ସ୍ୱତଃସ୍ପୁର୍ତ୍ତ ଭାବରେ ଘଟିଥାଏ ବା କ୍ୱଚିତ୍ ଏହା ପିତାମାତାଙ୍କଠାରୁ ଉତ୍ତରାଧିକାରୀ ସୂତ୍ରରେ ଆସିଥାଇପାରେ ।  ଏଥିରେ ପିଏଏକ୍ସ୬ ଏବଂ ଡବ୍ଲ୍ୟୁଟି୧ ସମେତ ପରସ୍ପର ପାଖରେ ବସିଥିବା ଏକ ଜିନ  ଗୋଷ୍ଠୀର ଅଭାବ ଥାଏ ।  ଏଥିରେ ବିଡିଏନଏଗଫ ଜିନ୍ ଜଡିତ ହେଲେ ମୋଟାପଣ ହୁଏ ।  ଜନ୍ମ ସମୟରେ ପରୀକ୍ଷା ଉପରେ ଆଧାର କରି ରୋଗ ନିର୍ଣ୍ଣୟ କରାଯାଏ ଏବଂ କାରିଓଟାଇପିଂଦ୍ୱାରା ନିଶ୍ଚିତ କରାଯାଇଥାଏ । 
ଲକ୍ଷଣ ଉପରେ ଚିକିତ୍ସା ଆଧାରିତ ହୁଏ ।  ୱିମସ ଅର୍ବୁଦ ସାଧାରଣତଃ ଅସ୍ତ୍ରୋପଚାର ଏବଂ କେମୋଥେରାପିଦ୍ୱା

In [91]:
print(extract_txt('ଛାଞ୍ଚ:୨୦୨୨_ଫିଫା_ବିଶ୍ୱକପ୍'))

୨୦୨୨ ଫିଫା ବିଶ୍ୱକପ୍ ହେଉଛି ଏକ ଆନ୍ତର୍ଜାତୀୟ ଫୁଟବଲ୍ ଟୁର୍ଣ୍ଣାମେଣ୍ଟ ଯାହା ଫିଫା ସଦସ୍ୟ ସଙ୍ଗଠନର ପୁରୁଷ ଜାତୀୟ ଦଳ ଦ୍ୱାରା ପ୍ରତିଦ୍ୱନ୍ଦ୍ୱିତା କରାଯାଏ । ୨୨ ନଭେମ୍ବରରୁ ୧୮ ଡିସେମ୍ବର ୨୦୨୨ ପର୍ଯ୍ୟନ୍ତ କତାରରେ ୨୨ ତମ ଫିଫା ବିଶ୍ୱକପ୍ ଅନୁଷ୍ଠିତ ହେଉଛି; ଆରବ ଜଗତ ଏବଂ ମୁସଲିମ ଜଗତରେ ଏହା ପ୍ରଥମ ବିଶ୍ୱକପ ଏବଂ ୨୦୨୨ରେ ଦକ୍ଷିଣ କୋରିଆ ଓ ଜାପାନର ଟୁର୍ନାମେଣ୍ଟ ପରେ ଏସିଆରେ ଅନୁଷ୍ଠିତ ହେଉଛି ।
ସର୍ବମୋଟ ୩୨ଟି ଅଂଶଗ୍ରହଣକାରୀ ଦଳ ଏହି ଟୁର୍ନାମେଣ୍ଟରେ ଭାଗନେଉଛନ୍ତି। କତାରର ଗରମ ଜଳବାୟୁ  ପାଇଁ ନଭେମ୍ବର ଏବଂ ଡିସେମ୍ବର ମାସରେ ଏହି ବିଶ୍ୱକପ୍ ଅନୁଷ୍ଠିତ ହେଉଛି ।ଏହା 29 ଦିନ ମଧ୍ୟରେ ପାଞ୍ଚଟି ସହରର ଆଠଟି ସ୍ଥାନରେ 64 ଟି ମ୍ୟାଚ୍ ଖେଳାଯିବ । କାତାର ଜାତୀୟ ଫୁଟବଲ୍ ଦଳ ସ୍ୱୟଂଚାଳିତ ଭାବରେ ଇଭେଣ୍ଟରେ ପ୍ରବେଶ କରିଥିଲା, ସେମାନଙ୍କର ପ୍ରଥମ ବିଶ୍ୱକପ୍, 31ଟି ଦଳ ସହିତ ଯୋଗ୍ୟତା ପ୍ରକ୍ରିୟା ଦ୍ୱାରା ସ୍ଥିର କରାଯାଇଥିଲା । କତାର ସମସ୍ତ ତିନୋଟି ଗ୍ରୁପ୍ ମ୍ୟାଚ୍ ହାରିଗଲା; ପ୍ରତ୍ୟେକ ଖେଳ ହରାଇବାରେ ସେମାନେ ପ୍ରଥମ ହୋଷ୍ଟ ହୋଇଥିଲେ,  ଏବଂ ଦ୍ୱିତୀୟ ହୋଷ୍ଟ ( 2010 ରେ ଦକ୍ଷିଣ ଆଫ୍ରିକା ପରେ) ପ୍ରଥମ ପର୍ଯ୍ୟାୟରେ ଅଗ୍ରଗତି ନକରିବା । 
ଟୁର୍ନାମେଣ୍ଟ ଫାଇନାଲର ପ୍ରଥମ ରାଉଣ୍ଡରେ ଦଳଗୁଡିକ ଚାରୋଟି ଦଳର ଆଠଟି ରାଉଣ୍ଡ-ରୋବିନ ଗ୍ରୁପରେ ପଏଣ୍ଟ ପାଇଁ ପ୍ରତିଯୋଗୀତା କରିଥିଲେ ଏବଂ ପ୍ରତ୍ୟେକ ଗୋଷ୍ଠୀର ଶ୍ରେଷ୍ଠ 

In [92]:
print(extract_txt('୨୦୧୮_ପୁରୁଷ_ହକି_ବିଶ୍ୱକପ'))

ପୁରୁଷ ହକି ବିଶ୍ୱକପର ୧୪ତମ ସଂସ୍କରଣ ଭୁବନେଶ୍ୱରରେସ୍ଥିତ କଳିଙ୍ଗ ଷ୍ଟାଡିୟମରେ ଅନୁଷ୍ଟିତ ହୋଇଥିଲା । ଏହି ପ୍ରତିଯୋଗିତା ୨୮ ନଭେମ୍ବରରୁ ୧୬ ଡିସେମ୍ବର ୨୦୧୮ ପର୍ଯ୍ୟନ୍ତ ଚାଲିଥିଲା  । ଏହାର ଉଦଘାଟନ ଉତ୍ସବ ୨୭ ନଭେମ୍ବରରେ ଜାକ ଯମକର ସହ ସମାପ୍ତ ହୋଇଛି । ଉଦଘାଟନୀ ଉତ୍ସବରେ ଶାହରୁଖ ଖାନ, ମାଧୁରୀ ଦୀକ୍ଷିତ ଏବଂ ଏ.ଆର ରହମାନ ଯୋଗ ଦେଇଥିଲେ । ଏଥି ସହ ଭାରତର ସବୁଠାରୁ ବଡ଼ ଡ୍ରୋଣ ଉଡାଣ ପ୍ରଦର୍ଶିତ ହୋଇଥିଲା ।ଅଂଶଗ୍ରହଣ କରୁଥିବା ୧୬ ଜଣ ଦଳକୁ ଚାରୋଟି ବିଭାଗରେ ବଣ୍ଟନ କରାଯାଇଅଛି । ପ୍ରତ୍ୟେକ ଦଳ ନିଜ ବିଭାଗରେ ଥିବା ଅନ୍ୟ ଦଳଗୁଡ଼ିକ ସହ ଥରେ ଥରେ ଖେଳିବେ । ପ୍ରତି ବିଭାଗରେ ପ୍ରଥମ ସ୍ଥାନ ଦାଖଲ କରିଥିବା ଦଳ କ୍ୱାର୍ଟର ଫାଇନାଲକୁ ଯିବେ । ଅନ୍ୟପକ୍ଷରେ ପ୍ରତି ବିଭାଗରେ ଦ୍ୱିତୀୟ ଏବଂ ତୃତୀୟ ସ୍ଥାନ ଅଧିକାର କରିଥିବା ଦଳଗୁଡ଼ିକ କ୍ରସ ଓଭର ଖେଳିବେ । ସେଠାରେ ଏକ ସିଙ୍ଗଲ ଏଲିମିନେସନ ପ୍ରତିଯୋଗିତା ହେବ । ଯେଉଁଠାରେ ହାରିଲାମାତ୍ରକେ ବିଶ୍ୱକପ ଦୌଡରୁ ଦଳଟି ବାଦ ପଡ଼ିଯିବ । ବିଶ୍ୱ ହକିର ୧୬ଟି ରାଷ୍ଟ୍ରକୁ ନେଇ ୧୯ ଦିନ ଧରି ଖେଳାଯିବାକୁ ଥିବା ପ୍ରତିଯୋଗିତାରେ ମୋଟ ୩୬ଟି ମ୍ୟାଚ ଖେଳାଯିବ। ନଭେମ୍ବର ୨୮ରୁ ଡିସେମ୍ବର ୯ ପର୍ଯ୍ୟନ୍ତ ମୋଟ ୨୪ଟି ଲିଗ୍‍ ମ୍ୟାଚ ଖେଳାଯିବ। ଡିସେମ୍ବର ୧୦ ଓ ୧୧ରେ ୪ଟି କ୍ରସ ଓଭର ମ୍ୟାଚ ଖେଳାଯିବାବେଳେ ୧୨ ଓ ୧୩ରେ କ୍ୱାର୍ଟର ଫାଇନାଲ ମ୍ୟାଚ ଖେଳାଯିବ। ୧୫ରେ ୨ଟି ସେମିଫାଇନାଲ ଓ ୧୬ରେ 

In [93]:
print(extract_txt('10th_Lok_Sabha'))

ଭାରତୀୟ ସଂସଦର ତଳ ସଦନ ଲୋକ ସଭାର ୧୦ମ ଲୋକ ସଭାପାଇଁ ୧୯୯୧ ମସିହାରେ ସାଧାରଣ ଅନୁଷ୍ଠିତ ହୋଇଥିଲା । ଏହି ଲୋକ ସଭାର କାର୍ଯ୍ୟକାଳ ଥିଲା ୨୦ ଜୁନ ୧୯୯୧ରୁ ୧୦ ମଇ ୧୯୯୬ । ଏହି ଲୋକ ସଭା ପାଇଁ ୧୯୯୧ ମସିହାରେ ମଇ-ଜୁନ ମାସରେ ନିର୍ବାଚନ ଅନୁଷ୍ଠିତ ହୋଇଥିଲା । ୧୯୯୧ର ସାଧାରଣ ନିର୍ବାଚନ ପରେ ରାଜ୍ୟ ସଭାର ୪ ଜଣ ସଦସ୍ୟ ଲୋକ ସଭାକୁ ମନୋନୀତ ହୋଇଥିଲେ ।
ଏହି ଲୋକ ସଭାରେ କଂଗ୍ରେସ ଦଳ ସମୁଦାୟ ୨୪୭ଟି ଆସନ ଗ୍ରହଣ କରିଥିଲା, ଯାହାକି ପୂର୍ବ ଲୋକ ସଭା ଅର୍ଥାତ ୯ମ ଲୋକ ସଭାଠାରୁ ୪୭ଟି ଆସନ ଅଧିକ ଥିଲା । ଏହି ନିର୍ବାଚନ ପରେ କଂଗ୍ରେସ ଦଳ ସଂଖ୍ୟାଗରିଷ୍ଠତା ହାସଲ କରିବା ପରେ ପି. ଭି. ନରସିଂହ ରାଓ ଭାରତର ପ୍ରଧାନମନ୍ତ୍ରୀ ଭାବରେ ଦାୟିତ୍ୱ ନେଇଥିଲେ । ତାଙ୍କର କାର୍ଯ୍ୟକାଳ ଥିଲା ୨୧ ଜୁନ ୧୯୯୧ରୁ ୧୬ ମଇ ୧୯୯୬ । ଏହାପରେ ୧୧ଶ ଲୋକ ସଭା ୧୯୯୬ ମସିହା ସାଧାରଣ ନିର୍ବାଚନ ପରେ ୧୫ ମଇ ୧୯୯୬ରେ ଗଠିତ ହୋଇଥିଲା ।ବାଚସ୍ପତି:
ଶିବରାଜ ପାଟିଲ (୧୦ ଜୁଲାଇ ୧୯୯୧ରୁ ୨୨ ମଇ ୧୯୯୬)
ଉପ-ବାଚସ୍ପତି:
ଏସ୍. ମଲ୍ଲିକାର୍ଜୁନାୟ (୧୩ ଅଗଷ୍ଟ ୧୯୯୧ରୁ ୧୦ ମଇ ୧୯୯୬)
ମହା ସଚିବ
କେ ସି ରସ୍ତୋଗି (୨୦ ଜୁନ ୧୯୯୧ରୁ ୩୧ ଡିସେମ୍ବର ୧୯୯୧)
ସି କେ ଜୈନ (୧ ଜାନୁଆରୀ ୧୯୯୨ରୁ ୩୧ ମଇ ୧୯୯୪)
ଆର୍. ସି. ଭରଦ୍ୱାଜ (୩୧ ମଇ ୧୯୯୪ରୁ ୩୧ ଡିସେମ୍ବର ୧୯୯୫)
ଏସz. ଏନ୍. ମିଶ୍ର (୧ ଜାନୁଆରୀ ୧୯୯୬ରୁ ୧୦ ମଇ ୧୯୯୬)ବିଭିନ୍ନ ରାଜନୈତିକ ଦଳରୁ ୧୦ମ ଲୋକ ସଭାକୁ ନିର୍ବ

In [94]:
print(extract_txt('Electric_field'))
print('-----------')

ବୈଦୁତିକ କ୍ଷେତ୍ର ହେଉଛି ବିଦ୍ୟୁତ୍-ଚୁମ୍ବକୀୟ କ୍ଷେତ୍ରର ଏକ ଅଂଶ ବିଶେଷ । ଏହା ଏକ vector field ଅଟେ ଏବଂ ମାକ୍ସୱେଲଙ୍କ ସମୀକରଣର ବର୍ଣ୍ଣନା ଅନୁସାରେ ଏହା ବୈଦୁତିକ ଚାର୍ଜ କିମ୍ବା time-varying ଚୁମ୍ବକୀୟ କ୍ଷେତ୍ରଦ୍ୱାରା ସୃଷ୍ଟି ହୋଇଥାଏ । ବୈଦୁତିକ କ୍ଷେତ୍ର ସମ୍ପର୍କରେ ଧାରଣା ସର୍ବପ୍ରଥମେ ମାଇକେଲ ଫରାଡ଼େ ଦେଇଥିଲେ । 
-----------


In [95]:
links = df['links'].to_list()
len(links) == len(df)

True

In [96]:
get_txt_from_url = lambda url:  partial(extract_txt, urlparse(url).path.split('/wiki/')[1])
get_txt_from_url(links[0])()

"ବାସ୍ତବ ସଂଖ୍ୟା ସମୂହରେ ଶୁନ୍ୟ (୦)  ହେଉଛି ଏକ ଯୁଗ୍ମ ସଂଖ୍ୟା ।ଶୁନ୍ୟଦ୍ୱାରା ଯେ କୌଣସି ସଂଖ୍ୟାକୁ ଗୁଣନ କଲେ, ଗୁଣଫଳ ସର୍ବଦା ଶୁନ୍ୟ ହୁଏ ।\nଶୁନ୍ୟଦ୍ୱାରା ଭାଗଫଳ ଏକ ଅସମ୍ଭବ ପ୍ରକ୍ରିୟା ।\nଯେ କୌଣସି ସଂଖ୍ୟାରେ ଶୁନ୍ୟ (୦) ଯୋଗ କଲେ ମୂଳ ସଂଖ୍ୟାର ମୂଲ୍ୟରେ ପରିବର୍ତ୍ତନ ହୁଏ ନାହିଁ । ଅତଏବ ଶୁନ୍ୟ (୦)କୁ ଯୋଗାତ୍ମକ ଅଭେଦ କୁହାଯାଏ ।'"

In [61]:
texts = parallel(extract_txt, links, progress=True)

text='ଭଲପାଏ ତ\u200cତେ ୧୦୦ରୁ ୧୦୦, ୨୦୧୫ରେ ମୁକ୍ତିଲାଭ କରିଥିବା ଓଡ଼ିଆ କଥାଚିତ୍ର । ଏହି କଥାଚିତ୍ରର ପ୍ରଯୋଜନା କରିଥିଲେ ସୀତାରାମ ଅଗ୍ର\u200cୱାଲ । ରଣଜିତ ପଟ୍ଟନାୟକ ଏହି କଥାଚିତ୍ରର ସଂଳାପ ଲେଖିଥିଲେ । ନିର୍ଦ୍ଦେଶକ ସୁଧାଂଶୁ ମୋହନ ସାହୁ ଏହି କଥାଚିତ୍ରଟିର ନିର୍ଦ୍ଦେଶନା ଦେଇଥିଲେ । ସଙ୍ଗୀତ ନିର୍ଦ୍ଦେଶକ ବିକାଶ ଦାସ ଏହି ଚଳଚ୍ଚିତ୍ରଟିରେ ସ୍ୱର ସଂଯୋଜନା କରିଥିଲେ । ଗୀତିକାର ଶ୍ରୀକାନ୍ତ ଗୌତମ ଏହି କଥାଚିତ୍ରର ଗୀତଗୁଡ଼ିକ ରଚନା କରିଥିଲେ । ନମିତା ଅଗ୍ରୱାଲ, ସୌରିନ ଭଟ୍ଟ, ବାବୁଶାନ୍ ଆଦି କଣ୍ଠଶିଳ୍ପୀମାନେ କଥାଚିତ୍ରଟିରେ କଣ୍ଠଦାନ କରିଥିଲେ ।\nଏହି ଚଳଚ୍ଚିତ୍ରର ମୁଖ୍ୟ ଭୂମିକାରେ ଅଭିନୟ କରିଥିଲେ ବାବୁଶାନ୍ । ବାବୁଙ୍କ ସ\u200cହ ନାୟିକା ଭୂମିକାରେ ଅଭିନୟ କରିଥିଲେ ଶୀତଲ ପାତ୍ର ଓ ସୂର୍ଯ୍ୟମୟୀ ମହାପାତ୍ର । ସୂର୍ଯ୍ୟମୟୀ ସାର୍ଥକ ଟିଭିରେ ପ୍ରସାରିତ କାର୍ଯ୍ୟକ୍ରମ ମୁ ବି ହିରୋଇନ ହେବିରେ ଅଂଶଗ୍ରହଣ କରିଥିଲେ ଏବଂ ବିଜୟୀ ହେବାପରେ ଏହି ଚଳଚ୍ଚିତ୍ରଟିରେ ତାଙ୍କୁ ଅଭିନୟର ସୁଯୋଗ ମିଳିଥିଲା । ହରିହର ମହାପାତ୍ର, ପୃଥିରାଜ, ଅମିତ ପାଣି, ଦେବାଶିଷ ପାତ୍ର ଆଦି କଳାକାରମାନେ ଏହି ଚଳଚ୍ଚିତ୍ରରେ ବିଭିନ୍ନ ଭୂମିକାରେ ଅଭିନୟ କରିଥିଲେ ।\n\n\n== ଅଭିନୟ ==\nବାବୁଶାନ୍\nଶୀତଲ ପାତ୍ର\nହରିହର ମହାପାତ୍ର\nସୂର୍ଯ୍ୟମୟୀ ମହାପାତ୍ର\nପୃଥିରାଜ ନାୟକ\nଅମିତ ପାଣି\nଦେବାଶିଷ ପାତ୍ର\nସ୍ମରଣୀକା ପ୍ରିୟଦର୍ଶିନୀ

In [ ]:
data = {
    'link': links,
    'text': texts
}
data_df = pd.DataFrame.from_dict(data)
data_df.head()
data_df.to_csv("odia_wiki.csv", index=False )